## Setup

In [2]:
import json
import re
import datetime
from collections import OrderedDict

import pandas as pd
import pandas.io.json as pj
from IPython.core.debugger import set_trace

import tbap.api as api
import tbap.server as server

import auth

In [3]:
key = auth.key
username = auth.username
sn = api.Session(username, key)
snj = api.Session(username, key, data_format="json")

# Display floating point numbers in fixed format within dataframes.
pd.options.display.float_format = '{:f}'.format

### get_matches()

In [16]:
sn.time_zone = "America/Chicago"
matches = api.get_matches(sn, "2019roe")

In [17]:
matches.head(24)

alliance  surrogate  score  disqualified  \
key           team_key                                            
2019roe_f1m1  frc6829      blue      False    107         False   
              frc148       blue      False    107         False   
              frc3847      blue      False    107         False   
              frc1658       red      False     49         False   
              frc5411       red      False     49         False   
              frc2485       red      False     49         False   
2019roe_f1m2  frc2907      blue      False    102         False   
              frc148       blue      False    102         False   
              frc3847      blue      False    102         False   
              frc1658       red      False     85         False   
              frc5411       red      False     85         False   
              frc3737       red      False     85         False   
2019roe_qf1m1 frc4336      blue      False     83         False   
              frc2655      blue      False     83         False   
              frc2642      blue      False     83         False   
              frc3647       red      False     77         False   
              frc294        red      False     77         False   
              frc1410       red      False     77         False   
2019roe_qf1m2 frc4336      blue      False     75         False   
              frc2655      blue      False     75         False   
              frc2642      blue      False     75         False   
              frc3647       red      False     83         False   
              frc294        red      False     83         False   
              frc1410       red      False     83         False   

                                     actual_time comp_level event_key  \
key           team_key                                                  
2019roe_f1m1  frc6829  2019-04-20 12:37:02-05:00          f   2019roe   
              frc148   2019-04-20 12:37:02-05:00          f   2019roe   
              frc3847  2019-04-20 12:37:02-05:00          f   2019roe   
              frc1658  2019-04-20 12:37:02-05:00          f   2019roe   
              frc5411  2019-04-20 12:37:02-05:00          f   2019roe   
              frc2485  2019-04-20 12:37:02-05:00          f   2019roe   
2019roe_f1m2  frc2907  2019-04-20 12:50:01-05:00          f   2019roe   
              frc148   2019-04-20 12:50:01-05:00          f   2019roe   
              frc3847  2019-04-20 12:50:01-05:00          f   2019roe   
              frc1658  2019-04-20 12:50:01-05:00          f   2019roe   
              frc5411  2019-04-20 12:50:01-05:00          f   2019roe   
              frc3737  2019-04-20 12:50:01-05:00          f   2019roe   
2019roe_qf1m1 frc4336  2019-04-20 09:33:04-05:00         qf   2019roe   
              frc2655  2019-04-20 09:33:04-05:00         qf   2019roe   
              frc2642  2019-04-20 09:33:04-05:00         qf   2019roe   
              frc3647  2019-04-20 09:33:04-05:00         qf   2019roe   
              frc294   2019-04-20 09:33:04-05:00         qf   2019roe   
              frc1410  2019-04-20 09:33:04-05:00         qf   2019roe   
2019roe_qf1m2 frc4336  2019-04-20 10:06:26-05:00         qf   2019roe   
              frc2655  2019-04-20 10:06:26-05:00         qf   2019roe   
              frc2642  2019-04-20 10:06:26-05:00         qf   2019roe   
              frc3647  2019-04-20 10:06:26-05:00         qf   2019roe   
              frc294   2019-04-20 10:06:26-05:00         qf   2019roe   
              frc1410  2019-04-20 10:06:26-05:00         qf   2019roe   

                        match_number          post_result_time  \
key           team_key                                           
2019roe_f1m1  frc6829              1 2019-04-20 12:40:18-05:00   
              frc148               1 2019-04-20 12:40:18-05:00   
              frc3847              1 2019-04-20 12:40:18-05:00   
              frc1658              1 2019-04-20 12:

In [18]:
matches.loc["2019roe_qf3m1"].at["frc1318", "score"]

82

In [19]:
matches.xs("frc1318", level=1).head()

,alliance,surrogate,score,disqualified,actual_time,comp_level,event_key,match_number,post_result_time,predicted_time,...,preMatchLevelRobot3,rp,sandStormBonusPoints,techFoulCount,teleopPoints,topLeftRocketFar,topLeftRocketNear,topRightRocketFar,topRightRocketNear,totalPoints
key,,,,,,,,,,,,,,,,,,,,,
2019roe_qf3m1,blue,False,82,False,2019-04-20 09:52:40-05:00,qf,2019roe,1,2019-04-20 09:56:46-05:00,2019-04-20 09:53:00-05:00,...,HabLevel2,0,15,0,67,None,None,None,None,82
2019roe_qf3m2,blue,False,68,False,2019-04-20 10:26:47-05:00,qf,2019roe,2,2019-04-20 10:30:50-05:00,2019-04-20 10:28:02-05:00,...,HabLevel2,0,15,0,53,Panel,None,None,None,68
2019roe_qm111,blue,False,63,False,2019-04-19 17:47:24-05:00,qm,2019roe,111,2019-04-19 17:50:17-05:00,2019-04-19 17:47:33-05:00,...,HabLevel1,1,9,0,54,None,None,None,None,63
2019roe_qm20,red,False,77,False,2019-04-18 11:17:59-05:00,qm,2019roe,20,2019-04-18 11:21:37-05:00,2019-04-18 11:19:12-05:00,...,HabLevel1,3,15,0,52,None,None,None,None,77
2019roe_qm27,blue,False,69,False,2019-04-18 13:39:02-05:00,qm,2019roe,27,2019-04-18 13:42:32-05:00,2019-04-18 13:38:40-05:00,...,HabLevel1,2,12,0,54,Panel,None,PanelAndCargo,None,69


In [20]:
matches.shape

(776, 64)

In [22]:
matches = api.get_matches(sn, "2019roe", response="simple")

In [12]:
matches.head(12)

alliance  surrogate  score  disqualified  \
key          team_key                                            
2017tur_f1m1 frc2046      blue      False    503         False   
             frc1318      blue      False    503         False   
             frc1595      blue      False    503         False   
             frc1296       red      False    271         False   
             frc2473       red      False    271         False   
             frc1323       red      False    271         False   
2017tur_f1m2 frc2046      blue      False    461         False   
             frc1318      blue      False    461         False   
             frc1595      blue      False    461         False   
             frc1296       red      False    424         False   
             frc2473       red      False    424         False   
             frc1323       red      False    424         False   

                                    actual_time comp_level event_key  \
key          team_key                                                  
2017tur_f1m1 frc2046  2017-04-22 12:18:21-05:00          f   2017tur   
             frc1318  2017-04-22 12:18:21-05:00          f   2017tur   
             frc1595  2017-04-22 12:18:21-05:00          f   2017tur   
             frc1296  2017-04-22 12:18:21-05:00          f   2017tur   
             frc2473  2017-04-22 12:18:21-05:00          f   2017tur   
             frc1323  2017-04-22 12:18:21-05:00          f   2017tur   
2017tur_f1m2 frc2046  2017-04-22 12:35:04-05:00          f   2017tur   
             frc1318  2017-04-22 12:35:04-05:00          f   2017tur   
             frc1595  2017-04-22 12:35:04-05:00          f   2017tur   
             frc1296  2017-04-22 12:35:04-05:00          f   2017tur   
             frc2473  2017-04-22 12:35:04-05:00          f   2017tur   
             frc1323  2017-04-22 12:35:04-05:00          f   2017tur   

                       match_number            predicted_time  set_number  \
key          team_key                                                       
2017tur_f1m1 frc2046              1 2017-04-22 12:19:01-05:00           1   
             frc1318              1 2017-04-22 12:19:01-05:00           1   
             frc1595              1 2017-04-22 12:19:01-05:00           1   
             frc1296              1 2017-04-22 12:19:01-05:00           1   
             frc2473              1 2017-04-22 12:19:01-05:00           1   
             frc1323              1 2017-04-22 12:19:01-05:00           1   
2017tur_f1m2 frc2046              2 2017-04-22 12:34:01-05:00           1   
             frc1318              2 2017-04-22 12:34:01-05:00           1   
             frc1595              2 2017-04-22 12:34:01-05:00           1   
             frc1296              2 2017-04-22 12:34:01-05:00           1   
             frc2473              2 2017-04-22 12:34:01-05:00           1   
             frc1323              2 2017-04-22 12:34:01-05:00           1   

                                           time winning_alliance  
key          team_key                                             
2017tur_f1m1 frc2046  2017-04-22 12:00:00-05:00             blue  
             frc1318  2017-04-22 12:00:00-05:00             blue  
             frc1595  2017-04-22 12:00:00-05:00             blue  
             frc1296  2017-04-22 12:00:00-05:00             blue  
             frc2473  2017-04-22 12:00:00-05:00             blue  
             frc1323  2017-04-22 12:00:00-05:00             blue  
2017tur_f1m2 frc2046  2017-04-22 12:13:00-05:00             blue  
             frc1318  2017-04-22 12:13:00-05:00             blue  
             frc1595  2017-04-22 12:13:00-05:00             blue  
             frc1296  2017-04-22 12:13:00-05:00             blue  
             frc2473  2017-04-22 12:13:00-05:00             blue  
             frc1323  2017-04-22 12:13:00-05:00             blue

In [23]:
matches.shape

(776, 12)

In [26]:
matches = api.get_matches(sn, "2019roe", response="keys")

In [27]:
matches.head()

,key
0,2019roe_f1m1
1,2019roe_f1m2
2,2019roe_qf1m1
3,2019roe_qf1m2
4,2019roe_qf1m3


In [28]:
matches = api.get_matches(sn, "2019roe", "frc1318")

In [29]:
matches.head()

alliance  surrogate  score  disqualified  \
key           team_key                                            
2019roe_qf3m1 frc1318      blue      False     82         False   
              frc2412      blue      False     82         False   
              frc3881      blue      False     82         False   
              frc6829       red      False     88         False   
              frc148        red      False     88         False   

                                     actual_time comp_level event_key  \
key           team_key                                                  
2019roe_qf3m1 frc1318  2019-04-20 09:52:40-05:00         qf   2019roe   
              frc2412  2019-04-20 09:52:40-05:00         qf   2019roe   
              frc3881  2019-04-20 09:52:40-05:00         qf   2019roe   
              frc6829  2019-04-20 09:52:40-05:00         qf   2019roe   
              frc148   2019-04-20 09:52:40-05:00         qf   2019roe   

                        match_number          post_result_time  \
key           team_key                                           
2019roe_qf3m1 frc1318              1 2019-04-20 09:56:46-05:00   
              frc2412              1 2019-04-20 09:56:46-05:00   
              frc3881              1 2019-04-20 09:56:46-05:00   
              frc6829              1 2019-04-20 09:56:46-05:00   
              frc148               1 2019-04-20 09:56:46-05:00   

                                  predicted_time  ...  preMatchLevelRobot3 rp  \
key           team_key                            ...                           
2019roe_qf3m1 frc1318  2019-04-20 09:53:00-05:00  ...            HabLevel2  0   
              frc2412  2019-04-20 09:53:00-05:00  ...            HabLevel2  0   
              frc3881  2019-04-20 09:53:00-05:00  ...            HabLevel2  0   
              frc6829  2019-04-20 09:53:00-05:00  ...            HabLevel2  0   
              frc148   2019-04-20 09:53:00-05:00  ...            HabLevel2  0   

                       sandStormBonusPoints  techFoulCount  teleopPoints  \
key           team_key                                                     
2019roe_qf3m1 frc1318                    15              0            67   
              frc2412                    15              0            67   
              frc3881                    15              0            67   
              frc6829                    15              0            64   
              frc148                     15              0            64   

                       topLeftRocketFar topLeftRocketNear topRightRocketFar  \
key           team_key                                                        
2019roe_qf3m1 frc1318              None              None              None   
              frc2412              None              None              None   
              frc3881              None              None              None   
              frc6829             Panel              None     PanelAndCargo   
              frc148              Panel              None     PanelAndCargo   

                       topRightRocketNear totalPoints  
key           team_key                                 
2019roe_qf3m1 frc1318                None          82  
              frc2412                None          82  
              frc3881                None          82  
              frc6829                None          88  
              frc148                 None          88  

[5 rows x 64 columns]

In [30]:
matches = api.get_matches(sn, "2019roe", "frc1318", response="simple")

In [31]:
matches[50:55]

alliance  surrogate  score  disqualified  \
key          team_key                                            
2019roe_qm73 frc1318      blue      False     50         False   
             frc2557       red      False     56         False   
             frc1369       red      False     56         False   
             frc7425       red      False     56         False   
2019roe_qm87 frc2485      blue      False     61         False   

                                    actual_time comp_level event_key  \
key          team_key                                                  
2019roe_qm73 frc1318  2019-04-19 11:02:43-05:00         qm   2019roe   
             frc2557  2019-04-19 11:02:43-05:00         qm   2019roe   
             frc1369  2019-04-19 11:02:43-05:00         qm   2019roe   
             frc7425  2019-04-19 11:02:43-05:00         qm   2019roe   
2019roe_qm87 frc2485  2019-04-19 14:30:30-05:00         qm   2019roe   

                       match_number            predicted_time  set_number  \
key          team_key                                                       
2019roe_qm73 frc1318             73 2019-04-19 11:02:24-05:00           1   
             frc2557             73 2019-04-19 11:02:24-05:00           1   
             frc1369             73 2019-04-19 11:02:24-05:00           1   
             frc7425             73 2019-04-19 11:02:24-05:00           1   
2019roe_qm87 frc2485             87 2019-04-19 14:34:08-05:00           1   

                                           time winning_alliance  
key          team_key                                             
2019roe_qm73 frc1318  2019-04-19 10:34:00-05:00              red  
             frc2557  2019-04-19 10:34:00-05:00              red  
             frc1369  2019-04-19 10:34:00-05:00              red  
             frc7425  2019-04-19 10:34:00-05:00              red  
2019roe_qm87 frc2485  2019-04-19 14:17:00-05:00              red

In [32]:
matches.shape

(72, 12)

In [33]:
matches = api.get_matches(sn, "2019roe", "frc1318", response="keys")

In [34]:
matches.head()

,key
0,2019roe_qf3m1
1,2019roe_qf3m2
2,2019roe_qm111
3,2019roe_qm20
4,2019roe_qm27


In [35]:
matches = api.get_matches(sn, team="frc1318", year=2019)

In [36]:
matches.query("score == -1")

,,alliance,surrogate,score,disqualified,actual_time,comp_level,event_key,match_number,post_result_time,predicted_time,...,preMatchLevelRobot3,rp,sandStormBonusPoints,techFoulCount,teleopPoints,topLeftRocketFar,topLeftRocketNear,topRightRocketFar,topRightRocketNear,totalPoints
key,team_key,,,,,,,,,,,,,,,,,,,,,


In [24]:
matches.tail(6)

alliance  surrogate  score  disqualified  \
key            team_key                                            
2017wasno_qm79 frc1318      blue      False    305         False   
               frc4077      blue      False    305         False   
               frc2976      blue      False    305         False   
               frc3070       red      False    259         False   
               frc4911       red      False    259         False   
               frc949        red      False    259         False   

                                      actual_time comp_level  event_key  \
key            team_key                                                   
2017wasno_qm79 frc1318  2017-03-26 13:24:20-05:00         qm  2017wasno   
               frc4077  2017-03-26 13:24:20-05:00         qm  2017wasno   
               frc2976  2017-03-26 13:24:20-05:00         qm  2017wasno   
               frc3070  2017-03-26 13:24:20-05:00         qm  2017wasno   
               frc4911  2017-03-26 13:24:20-05:00         qm  2017wasno   
               frc949   2017-03-26 13:24:20-05:00         qm  2017wasno   

                         match_number          post_result_time  \
key            team_key                                           
2017wasno_qm79 frc1318             79 2017-03-26 13:27:03-05:00   
               frc4077             79 2017-03-26 13:27:03-05:00   
               frc2976             79 2017-03-26 13:27:03-05:00   
               frc3070             79 2017-03-26 13:27:03-05:00   
               frc4911             79 2017-03-26 13:27:03-05:00   
               frc949              79 2017-03-26 13:27:03-05:00   

                                   predicted_time       ...         \
key            team_key                                 ...          
2017wasno_qm79 frc1318  2017-03-26 13:25:03-05:00       ...          
               frc4077  2017-03-26 13:25:03-05:00       ...          
               frc2976  2017-03-26 13:25:03-05:00       ...          
               frc3070  2017-03-26 13:25:03-05:00       ...          
               frc4911  2017-03-26 13:25:03-05:00       ...          
               frc949   2017-03-26 13:25:03-05:00       ...          

                         teleopFuelHigh teleopFuelLow teleopFuelPoints  \
key            team_key                                                  
2017wasno_qm79 frc1318         0.000000      0.000000         0.000000   
               frc4077         0.000000      0.000000         0.000000   
               frc2976         0.000000      0.000000         0.000000   
               frc3070        14.000000      0.000000         4.000000   
               frc4911        14.000000      0.000000         4.000000   
               frc949         14.000000      0.000000         4.000000   

                         teleopPoints  teleopRotorPoints  teleopTakeoffPoints  \
key            team_key                                                         
2017wasno_qm79 frc1318     230.000000          80.000000           150.000000   
               frc4077     230.000000          80.000000           150.000000   
               frc2976     230.000000          80.000000           150.000000   
               frc3070     184.000000          80.000000           100.000000   
               frc4911     184.000000          80.000000           100.000000   
               frc949      184.000000          80.000000           100.000000   

                         totalPoints      touchpadFar   touchpadMiddle  \
key            team_key                                                  
2017wasno_qm79 frc1318    305.000000  ReadyForTakeoff  ReadyForTakeoff   
               frc4077    305.000000  ReadyForTakeoff  ReadyForTakeoff   
               frc2976    305.000000  ReadyForTakeoff  ReadyForTakeoff   
               frc3070    259.000000             None  ReadyForTakeoff   
               frc4911    259.000000             None  ReadyForTakeoff   
               frc94

In [37]:
matches.shape

(432, 64)

In [38]:
matches = api.get_matches(sn, team="frc1318", year=2019, response="simple")

In [39]:
matches.head()

alliance  surrogate  score  disqualified  \
key             team_key                                            
2019pncmp_qf1m1 frc1983      blue      False     63         False   
                frc1318      blue      False     63         False   
                frc2928      blue      False     63         False   
                frc2910       red      False     98         False   
                frc2046       red      False     98         False   

                                       actual_time comp_level  event_key  \
key             team_key                                                   
2019pncmp_qf1m1 frc1983  2019-04-06 15:18:28-05:00         qf  2019pncmp   
                frc1318  2019-04-06 15:18:28-05:00         qf  2019pncmp   
                frc2928  2019-04-06 15:18:28-05:00         qf  2019pncmp   
                frc2910  2019-04-06 15:18:28-05:00         qf  2019pncmp   
                frc2046  2019-04-06 15:18:28-05:00         qf  2019pncmp   

                          match_number            predicted_time  set_number  \
key             team_key                                                       
2019pncmp_qf1m1 frc1983              1 2019-04-06 15:22:01-05:00           1   
                frc1318              1 2019-04-06 15:22:01-05:00           1   
                frc2928              1 2019-04-06 15:22:01-05:00           1   
                frc2910              1 2019-04-06 15:22:01-05:00           1   
                frc2046              1 2019-04-06 15:22:01-05:00           1   

                                              time winning_alliance  
key             team_key                                             
2019pncmp_qf1m1 frc1983  2019-04-06 15:15:00-05:00              red  
                frc1318  2019-04-06 15:15:00-05:00              red  
                frc2928  2019-04-06 15:15:00-05:00              red  
                frc2910  2019-04-06 15:15:00-05:00              red  
                frc2046  2019-04-06 15:15:00-05:00              red

In [40]:
matches = api.get_matches(sn, team="frc1318", year=2019, response="keys")

In [41]:
matches.head()

,key
0,2019pncmp_qf1m1
1,2019pncmp_qf1m2
2,2019pncmp_qm101
3,2019pncmp_qm110
4,2019pncmp_qm126


In [42]:
matches = api.get_matches(sn, match="2019pncmp_qf1m1")

In [43]:
matches

alliance  surrogate  score  disqualified  \
key             team_key                                            
2019pncmp_qf1m1 frc1983      blue      False     63         False   
                frc1318      blue      False     63         False   
                frc2928      blue      False     63         False   
                frc2910       red      False     98         False   
                frc2046       red      False     98         False   
                frc2907       red      False     98         False   

                                       actual_time comp_level  event_key  \
key             team_key                                                   
2019pncmp_qf1m1 frc1983  2019-04-06 15:18:28-05:00         qf  2019pncmp   
                frc1318  2019-04-06 15:18:28-05:00         qf  2019pncmp   
                frc2928  2019-04-06 15:18:28-05:00         qf  2019pncmp   
                frc2910  2019-04-06 15:18:28-05:00         qf  2019pncmp   
                frc2046  2019-04-06 15:18:28-05:00         qf  2019pncmp   
                frc2907  2019-04-06 15:18:28-05:00         qf  2019pncmp   

                          match_number          post_result_time  \
key             team_key                                           
2019pncmp_qf1m1 frc1983              1 2019-04-06 15:27:23-05:00   
                frc1318              1 2019-04-06 15:27:23-05:00   
                frc2928              1 2019-04-06 15:27:23-05:00   
                frc2910              1 2019-04-06 15:27:23-05:00   
                frc2046              1 2019-04-06 15:27:23-05:00   
                frc2907              1 2019-04-06 15:27:23-05:00   

                                    predicted_time  ...  preMatchLevelRobot3  \
key             team_key                            ...                        
2019pncmp_qf1m1 frc1983  2019-04-06 15:22:01-05:00  ...            HabLevel2   
                frc1318  2019-04-06 15:22:01-05:00  ...            HabLevel2   
                frc2928  2019-04-06 15:22:01-05:00  ...            HabLevel2   
                frc2910  2019-04-06 15:22:01-05:00  ...            HabLevel1   
                frc2046  2019-04-06 15:22:01-05:00  ...            HabLevel1   
                frc2907  2019-04-06 15:22:01-05:00  ...            HabLevel1   

                         rp sandStormBonusPoints  techFoulCount  teleopPoints  \
key             team_key                                                        
2019pncmp_qf1m1 frc1983   0                    9              0            44   
                frc1318   0                    9              0            44   
                frc2928   0                    9              0            44   
                frc2910   0                   15              1            80   
                frc2046   0                   15              1            80   
                frc2907   0                   15              1            80   

                         topLeftRocketFar topLeftRocketNear topRightRocketFar  \
key             team_key                                                        
2019pncmp_qf1m1 frc1983              None              None              None   
                frc1318              None              None              None   
                frc2928              None              None              None   
                frc2910             Panel              None             Panel   
                frc2046             Panel              None             Panel   
                frc2907             Panel              None             Panel   

                         topRightRocketNear totalPoints  
key             team_key                                 
2019pncmp_qf1m1 frc1983                None          63  
                frc1318                None          63  
                frc2928                None          63  
                frc2910                None          98  
                frc2046        

In [44]:
matches.shape

(6, 64)

In [46]:
sn.time_zone = "America/Los_Angeles"
matches = api.get_matches(sn, match="2019pncmp_qf1m1", response="simple")

In [47]:
matches

alliance  surrogate  score  disqualified  \
key             team_key                                            
2019pncmp_qf1m1 frc1983      blue      False     63         False   
                frc1318      blue      False     63         False   
                frc2928      blue      False     63         False   
                frc2910       red      False     98         False   
                frc2046       red      False     98         False   
                frc2907       red      False     98         False   

                                       actual_time comp_level  event_key  \
key             team_key                                                   
2019pncmp_qf1m1 frc1983  2019-04-06 13:18:28-07:00         qf  2019pncmp   
                frc1318  2019-04-06 13:18:28-07:00         qf  2019pncmp   
                frc2928  2019-04-06 13:18:28-07:00         qf  2019pncmp   
                frc2910  2019-04-06 13:18:28-07:00         qf  2019pncmp   
                frc2046  2019-04-06 13:18:28-07:00         qf  2019pncmp   
                frc2907  2019-04-06 13:18:28-07:00         qf  2019pncmp   

                          match_number            predicted_time  set_number  \
key             team_key                                                       
2019pncmp_qf1m1 frc1983              1 2019-04-06 13:22:01-07:00           1   
                frc1318              1 2019-04-06 13:22:01-07:00           1   
                frc2928              1 2019-04-06 13:22:01-07:00           1   
                frc2910              1 2019-04-06 13:22:01-07:00           1   
                frc2046              1 2019-04-06 13:22:01-07:00           1   
                frc2907              1 2019-04-06 13:22:01-07:00           1   

                                              time winning_alliance  
key             team_key                                             
2019pncmp_qf1m1 frc1983  2019-04-06 13:15:00-07:00              red  
                frc1318  2019-04-06 13:15:00-07:00              red  
                frc2928  2019-04-06 13:15:00-07:00              red  
                frc2910  2019-04-06 13:15:00-07:00              red  
                frc2046  2019-04-06 13:15:00-07:00              red  
                frc2907  2019-04-06 13:15:00-07:00              red